# Presentation du cas

# Import librairies

In [2]:
import pandas as pd
import numpy as np
import os
import sys
import tqdm as tqdm
import glob
import string
import matplotlib.pyplot as plt
from scipy.stats import rankdata
import warnings
from pandas_profiling import ProfileReport
warnings.filterwarnings('ignore')

# Telechargements donnees

In [3]:
data=pd.read_csv("../../data/sante/openfoodfacts_search.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9937 entries, 0 to 9936
Columns: 174 entries, code to carnitine_100g
dtypes: float64(123), int64(3), object(48)
memory usage: 13.2+ MB


# Data exploration

In [ ]:
profile = ProfileReport(data, title="Data exploration summary")
profile.to_widgets()

Summarize dataset:   0%|          | 0/187 [00:00<?, ?it/s]